In [1]:
from threading import Thread, Event
from time import sleep

import datetime
import paho.mqtt.client as mqtt
import paho.mqtt.publish as publish
import pywinctl as pwc

# Activity
def parse():
    title = None
    app = pwc.getActiveWindow().getAppName()
    
    browsers = ['Google Chrome', 'Safari']
    if app in browsers:
        title = pwc.getActiveWindowTitle()
        
        if app == 'Google Chrome':
            title = title.split(" - Google Chrome")[0]
        
    activity = app if title is None else title
    return activity

# Probe
TRANSMIT_TO = "paho/test/telemetry"
EVENT = Event()

def track():
    global EVENT
    while not EVENT.is_set():
        timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
        activity = parse()
        publish.single(TRANSMIT_TO, f"{timestamp}`{activity}", hostname=SERVER_HOST)
        sleep(10)

def start():
    global EVENT
    
    EVENT.clear()
    probe = Thread(name='probe', target=track)
    probe.start()

def stop():
    global EVENT
    
    EVENT.set()


# MQTT
SERVER_HOST = "operator.punchtime.app"
SERVER_PORT = 1883
KEEPALIVE = 60
LISTEN_TO = "punchtime/SO3emVMqcaeFbNM2pS6tynhvevC3"

def on_connect(client, userdata, flags, rc):
    print(f"Connected with result code {rc}")
    client.subscribe(LISTEN_TO)
    
def on_disconnect(client, userdata,  rc):
    print("Disconnected")  
    
def on_message(client, userdata, msg):
    state = msg.payload.decode("utf-8").lower()
    print(state)
    
    match state:
        case "on":
            start()
        case "off":
            stop()
        case "stop":
            client.loop_stop()
            client.disconnect()
        case _:
            pass

In [2]:
# if __name__ == '__main__':
client = mqtt.Client()
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.on_message = on_message
client.connect(SERVER_HOST, SERVER_PORT, KEEPALIVE)

client.loop_forever()

Connected with result code 0
off
on
off
Disconnected


KeyboardInterrupt: 